In [1]:
import numpy as np
import matplotlib.pyplot as plt

from mne import Epochs, create_info, events_from_annotations
from mne.io import concatenate_raws, read_raw_edf
from mne.datasets import eegbci
from mne.decoding import CSP
from mne.time_frequency import AverageTFR

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import LabelEncoder

In [2]:
import matplotlib
import matplotlib.pyplot as plt
import pathlib
import mne
import mne_bids
import mne
%matplotlib inline
from mne.preprocessing import (ICA, create_eog_epochs, create_ecg_epochs)


In [3]:
def get_raw_EEG(sid):
    raw_path = 'sub-0%.2d_task-memory_eeg.set'%(sid)
    raw = mne.io.read_raw_eeglab(raw_path, preload = True)
    return raw
def get_event(filename):
    events, event_id = mne.events_from_annotations(filename)
    return events, event_id

In [4]:
raw = get_raw_EEG(42)

raw.info

/Users/jouanchung/miniconda3/lib/python3.9/site-packages/pymatreader/utils.py:122: UserWarning: Complex objects (like classes) are not supported. They are imported on a best effort base but your mileage will vary.
  warn('Complex objects (like classes) are not supported. '
/var/folders/pb/psfryrqn1dz3hg7_n8k1p2n00000gn/T/ipykernel_694/1262175846.py:3: RuntimeWarning: Estimated head radius (0.1 cm) is below the 3rd percentile for infant head size. Check if the montage_units argument is correct (the default is "mm", but your channel positions may be in different units).
  raw = mne.io.read_raw_eeglab(raw_path, preload = True)
/var/folders/pb/psfryrqn1dz3hg7_n8k1p2n00000gn/T/ipykernel_694/1262175846.py:3: RuntimeWarning: The data contains 'boundary' events, indicating data discontinuities. Be cautious of filtering and epoching around these events.
  raw = mne.io.read_raw_eeglab(raw_path, preload = True)


<Info | 8 non-empty values
 bads: []
 ch_names: Fp1, Fz, F3, F7, FT9, FC5, FC1, C3, T7, TP9, CP5, CP1, Pz, P3, ...
 chs: 63 EEG
 custom_ref_applied: False
 dig: 66 items (3 Cardinal, 63 EEG)
 highpass: 0.0 Hz
 lowpass: 500.0 Hz
 meas_date: unspecified
 nchan: 63
 projs: []
 sfreq: 1000.0 Hz
>

In [5]:
sfreq = raw.info["sfreq"]

In [6]:
sfreq

1000.0

In [7]:
# get events 
events, event_id= get_event(raw)
# onlt get the events in need 
events_dict=event_id
new_events_dict = {key: events_dict[key] for key in events_dict.keys()
       & {'6013130','6009090',}}
new_events_dict 

Used Annotations descriptions: ['500105', '500109', '500113', '500205', '500209', '500213', '500305', '500309', '500313', '500405', '500409', '500413', '500505', '500509', '500513', '500609', '500613', '500709', '500713', '500809', '500813', '500909', '500913', '501013', '501113', '501213', '501313', '6001051', '6001091', '6001130', '6001131', '6002051', '6002090', '6002091', '6002130', '6002131', '6003050', '6003051', '6003090', '6003091', '6003130', '6003131', '6004051', '6004090', '6004091', '6004130', '6004131', '6005050', '6005051', '6005090', '6005091', '6005130', '6005131', '6006090', '6006091', '6006130', '6006131', '6007090', '6007091', '6007130', '6007131', '6008090', '6008091', '6008130', '6008131', '6009090', '6009091', '6009130', '6009131', '6010130', '6010131', '6011130', '6012130', '6013130', 'boundary']


{'6009090': 66, '6013130': 74}

In [8]:
event_id = new_events_dict 
event_id 

{'6009090': 66, '6013130': 74}

In [9]:
raw.pick_types(meg=False, eeg=True, stim=False, eog=False, exclude="bads")

NOTE: pick_types() is a legacy function. New code should use inst.pick(...).


<RawEEGLAB | sub-042_task-memory_eeg.set, 63 x 9603300 (9603.3 s), ~4.51 GB, data loaded>

In [10]:
# Assemble the classifier using scikit-learn pipeline
clf = make_pipeline(
    CSP(n_components=4, reg=None, log=True, norm_trace=False),
    LinearDiscriminantAnalysis(),
)
n_splits = 3  # for cross-validation, 5 is better, here we use 3 for speed
cv = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

In [11]:
tmin, tmax = -2, 4.000
n_cycles = 10.0  # how many complete cycles: used to define window size
min_freq = 4.0
max_freq = 25.0
n_freqs = 4  # how many frequency bins to use

In [12]:
# Assemble list of frequency range tuples
freqs = np.linspace(min_freq, max_freq, n_freqs)  # assemble frequencies
freq_ranges = list(zip(freqs[:-1], freqs[1:]))  # make freqs list of tuples

# Infer window spacing from the max freq and number of cycles to avoid gaps
window_spacing = n_cycles / np.max(freqs) / 2.0
centered_w_times = np.arange(tmin, tmax, window_spacing)[1:]
n_windows = len(centered_w_times)

# Instantiate label encoder
le = LabelEncoder()

In [ ]:
# init scores
tf_scores = np.zeros((n_freqs - 1, n_windows))

# Loop through each frequency range of interest
for freq, (fmin, fmax) in enumerate(freq_ranges):
    # Infer window size based on the frequency being used
    w_size = n_cycles / ((fmax + fmin) / 2.0)  # in seconds

    # Apply band-pass filter to isolate the specified frequencies
    raw_filter = raw.copy().filter(
        fmin, fmax, fir_design="firwin", skip_by_annotation="edge"
    )

    # Extract epochs from filtered data, padded by window size
    epochs = Epochs(
        raw_filter,
        events,
        event_id,
        tmin - w_size,
        tmax + w_size,
        proj=False,
        baseline=None,
        preload=True,
    )
    epochs.drop_bad()
    y = le.fit_transform(epochs.events[:, 2])

    # Roll covariance, csp and lda over time
    for t, w_time in enumerate(centered_w_times):
        # Center the min and max of the window
        w_tmin = w_time - w_size / 2.0
        w_tmax = w_time + w_size / 2.0

        # Crop data into time-window of interest
        X = epochs.copy().crop(w_tmin, w_tmax).get_data()

        # Save mean scores over folds for each frequency and time window
        tf_scores[freq, t] = np.mean(
            cross_val_score(estimator=clf, X=X, y=y, scoring="roc_auc", cv=cv), axis=0
        )

Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 4 - 8.2 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 4.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 3.00 Hz)
- Upper passband edge: 8.20 Hz
- Upper transition bandwidth: 2.05 Hz (-6 dB cutoff frequency: 9.22 Hz)
- Filter length: 1651 samples (1.651 s)



[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.4s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    2.5s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    3.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    4.8s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  63 out of  63 | elapsed:  1.9min finished


Not setting metadata
66 matching events found
No baseline correction applied
Using data from preloaded Raw for 66 events and 7279 original time points ...
0 bad epochs dropped
Computing rank from data with rank=None
    Using tolerance 7e-05 (2.2e-16 eps * 63 dim * 5e+09  max singular value)
    Estimated rank (mag): 63
    MAG: rank 63 computed from 63 data channels with 0 projectors
Reducing data rank from 63 -> 63
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 7.8e-05 (2.2e-16 eps * 63 dim * 5.5e+09  max singular value)
    Estimated rank (mag): 63
    MAG: rank 63 computed from 63 data channels with 0 projectors
Reducing data rank from 63 -> 63
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 7e-05 (2.2e-16 eps * 63 dim * 5e+09  max singular value)
    Estimated rank (mag): 63
    MAG: rank 63 computed from 63 data channels with 0 projectors
Reducing data rank from 63 -> 

    MAG: rank 63 computed from 63 data channels with 0 projectors
Reducing data rank from 63 -> 63
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 6.9e-05 (2.2e-16 eps * 63 dim * 4.9e+09  max singular value)
    Estimated rank (mag): 63
    MAG: rank 63 computed from 63 data channels with 0 projectors
Reducing data rank from 63 -> 63
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 9.7e-05 (2.2e-16 eps * 63 dim * 6.9e+09  max singular value)
    Estimated rank (mag): 63
    MAG: rank 63 computed from 63 data channels with 0 projectors
Reducing data rank from 63 -> 63
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 6.8e-05 (2.2e-16 eps * 63 dim * 4.9e+09  max singular value)
    Estimated rank (mag): 63
    MAG: rank 63 computed from 63 data channels with 0 projectors
Reducing data rank from 63 -> 63
Estimating covariance 

    MAG: rank 63 computed from 63 data channels with 0 projectors
Reducing data rank from 63 -> 63
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 6.7e-05 (2.2e-16 eps * 63 dim * 4.8e+09  max singular value)
    Estimated rank (mag): 63
    MAG: rank 63 computed from 63 data channels with 0 projectors
Reducing data rank from 63 -> 63
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 7.3e-05 (2.2e-16 eps * 63 dim * 5.2e+09  max singular value)
    Estimated rank (mag): 63
    MAG: rank 63 computed from 63 data channels with 0 projectors
Reducing data rank from 63 -> 63
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 6.8e-05 (2.2e-16 eps * 63 dim * 4.9e+09  max singular value)
    Estimated rank (mag): 63
    MAG: rank 63 computed from 63 data channels with 0 projectors
Reducing data rank from 63 -> 63
Estimating covariance 

    MAG: rank 63 computed from 63 data channels with 0 projectors
Reducing data rank from 63 -> 63
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 6.5e-05 (2.2e-16 eps * 63 dim * 4.7e+09  max singular value)
    Estimated rank (mag): 63
    MAG: rank 63 computed from 63 data channels with 0 projectors
Reducing data rank from 63 -> 63
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 8e-05 (2.2e-16 eps * 63 dim * 5.7e+09  max singular value)
    Estimated rank (mag): 63
    MAG: rank 63 computed from 63 data channels with 0 projectors
Reducing data rank from 63 -> 63
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 6.8e-05 (2.2e-16 eps * 63 dim * 4.8e+09  max singular value)
    Estimated rank (mag): 63
    MAG: rank 63 computed from 63 data channels with 0 projectors
Reducing data rank from 63 -> 63
Estimating covariance us

    MAG: rank 63 computed from 63 data channels with 0 projectors
Reducing data rank from 63 -> 63
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 7.1e-05 (2.2e-16 eps * 63 dim * 5e+09  max singular value)
    Estimated rank (mag): 63
    MAG: rank 63 computed from 63 data channels with 0 projectors
Reducing data rank from 63 -> 63
Estimating covariance using EMPIRICAL
Done.
Computing rank from data with rank=None
    Using tolerance 7.7e-05 (2.2e-16 eps * 63 dim * 5.5e+09  max singular value)
    Estimated rank (mag): 63
    MAG: rank 63 computed from 63 data channels with 0 projectors
Reducing data rank from 63 -> 63
Estimating covariance using EMPIRICAL
Done.
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 8.2 - 12 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 pas

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    1.5s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    4.3s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    5.2s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    6.5s remaining:    0.0s
[Parallel(n_jobs=1)]: Done  63 out of  63 | elapsed:  2.3min finished


In [ ]:
# Set up time frequency object
av_tfr = AverageTFR(
    create_info(["freq"], sfreq),
    tf_scores[np.newaxis, :],
    centered_w_times,
    freqs[1:],
    1,
)
# plot time frequency decoding scores
chance = np.mean(y)  # set chance level to white in the plot
av_tfr.plot([0], vmin=chance, title="Time-Frequency Decoding Scores", cmap=plt.cm.Reds)

In [ ]:
# check the shape of decoding score 
tf_scores=tf_scores[np.newaxis,:]
tf_scores.shape

In [ ]:
# save the decoding score of each particpants for grouping later 
with open('scores_s41_tf.npy','wb') as f:
    np.save(f,tf_scores)

In [ ]:
data = np.load('scores_s34_tf.npy')
data1 = np.load('scores_s35_tf.npy')
data2 = np.load('scores_s36_tf.npy')
data3 = np.load('scores_s38_tf.npy')
data4 = np.load('scores_s39_tf.npy')
data5 = np.load('scores_s40_tf.npy')
data6 = np.load('scores_s41_tf.npy')
data7 = np.load('scores_s43_tf.npy')
data8 = np.load('scores_s44_tf.npy')
data9 = np.load('scores_s45_tf.npy')
data10 = np.load('scores_s46_tf.npy')
data11 = np.load('scores_s47_tf.npy')
data12 = np.load('scores_s48_tf.npy')
data13 = np.load('scores_s49_tf.npy')
data14 = np.load('scores_s51_tf.npy')
data15= np.load('scores_s52_tf.npy')
data16= np.load('scores_s53_tf.npy')
data17 = np.load('scores_s54_con = np.dstack((data, data1,data2,data4,data5,data6,data7,data8,data9,data10,data11,data12,data13,data14,data15,data16))
.npy')
con2 = np.vstack((data, data1,data2,data4,data5,data6,data7,data8,data9,data10,data11,data12,data13,data14,data15,data16,data17))
con2

In [ ]:
numnew =np.mean(con2, axis=0)
new = numnew.reshape(1,3,20)
new

In [ ]:
# plot group 
av_tfr = AverageTFR(
    create_info(["freq"], sfreq),
    new,
    centered_w_times,
    freqs[1:],
    1,
)

#chance = np.mean(y)  # set chance level to white in the plot
#av_tfr.plot([0], title="Time-Frequency Decoding Scores", cmap=plt.cm.Reds)
av_tfr.plot([0], vmin=chance, title="Time-Frequency Decoding Scores", cmap=plt.cm.Reds)